In [3]:
import graphlab
import numpy
dataset = graphlab.SFrame.read_csv('train.csv', verbose=False)

[INFO] 1457110252 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/LoicMDIVAD/Lmdapp/products/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1457110252 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to ldivad@isep.fr and will expire on January 04, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-2089 - Server binary: /Users/LoicMDIVAD/Lmdapp/products/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1457110252.log
[INFO] GraphLab Server Version: 1.7.1


In [4]:
print dataset.num_rows()
dataset_2=dataset.dropna('Age',how="all")
print dataset_2.num_rows()

891
714


In [27]:
(round(20.26832872))

20.0

In [3]:
df_age_null = dataset[dataset['Age'] == None]
df_age =dataset.dropna('Age', how="all")
df_age_train, df_age_test = df_age.random_split(.8 ,seed=1)

In [4]:
print "Total number of passengers: \t",len(df_age)
print "Passengers with unknow age: \t",len(df_age_null)
print "Test passengers: \t\t",len(df_age_test)
print "Train passengers: \t\t",len(df_age_train)

Total number of passengers: 	714
Passengers with unknow age: 	177
Test passengers: 		142
Train passengers: 		572


In [5]:
all_features = ['Survived', 'Pclass', 'Name','Sex', 'SibSp', 'Parch', 'Fare', 'Ticket', 'Embarked']

In [6]:
model_age = graphlab.linear_regression.create(df_age_train, target='Age', features=all_features,
                                              l1_penalty=0, l2_penalty=0.03, verbose=False,
                                              max_iterations=100)

In [7]:
model_age.evaluate(df_age_train)

{'max_error': 37.92236768527641, 'rmse': 3.470494329256639}

In [8]:
model_age.evaluate(df_age_test)

{'max_error': 39.06900808836291, 'rmse': 14.570481176795164}

In [9]:
df_compare = df_age_test['PassengerId', 'Sex', 'Age']
df_compare['predicted_age'] = model_age.predict(df_age_test)
df_compare.head(10)

PassengerId,Sex,Age,predicted_age
10,female,14.0,29.2651655006
11,female,4.0,22.2615033104
16,female,55.0,25.7884960048
22,male,34.0,25.3305795034
31,male,40.0,27.6621144695
41,female,40.0,22.513569487
42,female,27.0,26.3583206991
51,male,7.0,12.4495946345
53,female,49.0,31.7515408936
68,male,19.0,24.4184615939


In [31]:
def guess_age(ini_df):
    
    # Sampling
    df_age_null = ini_df[ini_df['Age'] == None]
    df_age = ini_df.dropna('Age', how="all")
    df_age_train, df_age_test = df_age.random_split(.8 ,seed=1)
    
    # allfeatures
    all_features = ['Survived', 'Pclass', 'Name','Sex', 'SibSp', 'Parch', 'Fare', 'Ticket', 'Embarked']
    
    #build the model
    model_age = graphlab.linear_regression.create(
        df_age_train, 
        target='Age', 
        features=all_features,
        l1_penalty=0, 
        l2_penalty=0.03, 
        verbose=False,
        max_iterations=100
    )
    
    df = df_age_null
    df['Age'] = model_age.predict(df)
    
    df['Age'] = df['Age'].apply(lambda y: int(round(y)))
    
    return df
    
    
    

In [32]:
dfgessed = guess_age(dataset) 
dfgessed .head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare
6,0,3,"Moran, Mr. James",male,25,0,0,330877,8.4583
18,1,2,"Williams, Mr. CharlesEugene ...",male,25,0,0,244373,13.0
20,1,3,"Masselmani, Mrs. Fatima",female,27,0,0,2649,7.225
27,0,3,"Emir, Mr. Farred Chehab",male,26,0,0,2631,7.225
29,1,3,"O'Dwyer, Miss. Ellen""Nellie"" ...",female,26,0,0,330959,7.8792
30,0,3,"Todoroff, Mr. Lalio",male,24,0,0,349216,7.8958
32,1,1,"Spencer, Mrs. WilliamAugustus (Marie Eugenie) ...",female,28,1,0,PC 17569,146.5208
33,1,3,"Glynn, Miss. Mary Agatha",female,26,0,0,335677,7.75
37,1,3,"Mamee, Mr. Hanna",male,26,0,0,2677,7.2292
43,0,3,"Kraeff, Mr. Theodor",male,26,0,0,349253,7.8958
